In [34]:
import numpy as np
import pandas as pd
import scipy.io as io
import math
import os
import scipy

In [87]:
def get_files(task:str, subdir = '\\results_zuco\\'):
    """
        Args: Task number ("task1", "task2", "task3") plus subdirectory
        Return: 12 matlab files (one per subject) for given task.
    """
    path = os.getcwd() + subdir + task
    files = [os.path.join(path,file) for file in os.listdir(path)[1:]]
    assert len(files) == 12, 'each task must contain 12 .mat files'
    return files

In [ ]:
getattr(data[0],data[0]._fieldnames[0])

In [131]:
def mk_dataframe(task:str, subject:int, word=False):
    """
        Args: Sentence level data for a single subject.
        Return: dataframe.
    """
    files = get_files(task)
    data = io.loadmat(files[subject], squeeze_me=True, struct_as_record=False)['sentenceData']
    fields = sorted(list(set([field for sent in data for field in sent._fieldnames if field != 'word'])))
    df = pd.DataFrame(index=range(1,len(data)+1), columns=[fields])
    for i, sent in enumerate(data):
        df.iloc[i] = [getattr(sent, field) if hasattr(sent, field) else 'NaN' for field in fields]
    return df

In [76]:
files_task1 = get_files('task1')
files_task2 = get_files('task2')
files_task3 = get_files('task3')

In [86]:
# index of the array `data` is the number of sentence
data = io.loadmat(files_task1[0], squeeze_me=True, struct_as_record=False)['sentenceData']

In [133]:
data = io.loadmat(files_task1[0], squeeze_me=True, struct_as_record=False)['sentenceData']

In [126]:
# example: print sentence
getattr(data[0],data[0]._fieldnames[0])

'Presents a good case while failing to provide a reason for us to care beyond the very basic dictums of human decency.'

In [138]:
fixations = data[0].allFixations

In [143]:
fixations.duration

array([ 28, 119,  73, 185, 122, 223, 142,  31, 100, 106, 145,  97,  91,
        79,  95,  82,  76,  79,  59,  95,  66,  77,  76,  90, 131, 104,
        58, 151], dtype=uint8)

In [144]:
# get word level data
word_data = data[0].word

In [153]:
data[0].content

'Presents a good case while failing to provide a reason for us to care beyond the very basic dictums of human decency.'

In [86]:
# index of the array `data` is the number of sentence
data = io.loadmat(files_task1[0], squeeze_me=True, struct_as_record=False)['sentenceData']

In [95]:
# example: get omission rate of first sentence
omission_rate = data[0].omissionRate
print(omission_rate)

0.22727272727272727


In [146]:
# get word level data
word_data = data[0].word

In [99]:
# get names of all word features
# index of the array `word_data` is the number of the word
print(word_data[0]._fieldnames)

['content', 'fixPositions', 'nFixations', 'meanPupilSize', 'rawEEG', 'rawET', 'FFD', 'FFD_pupilsize', 'FFD_t1', 'FFD_t2', 'FFD_a1', 'FFD_a2', 'FFD_b1', 'FFD_b2', 'FFD_g1', 'FFD_g2', 'FFD_t1_diff', 'FFD_t2_diff', 'FFD_a1_diff', 'FFD_a2_diff', 'FFD_b1_diff', 'FFD_b2_diff', 'FFD_g1_diff', 'FFD_g2_diff', 'TRT', 'TRT_pupilsize', 'TRT_t1', 'TRT_t2', 'TRT_a1', 'TRT_a2', 'TRT_b1', 'TRT_b2', 'TRT_g1', 'TRT_g2', 'TRT_t1_diff', 'TRT_t2_diff', 'TRT_a1_diff', 'TRT_a2_diff', 'TRT_b1_diff', 'TRT_b2_diff', 'TRT_g1_diff', 'TRT_g2_diff', 'GD', 'GD_pupilsize', 'GD_t1', 'GD_t2', 'GD_a1', 'GD_a2', 'GD_b1', 'GD_b2', 'GD_g1', 'GD_g2', 'GD_t1_diff', 'GD_t2_diff', 'GD_a1_diff', 'GD_a2_diff', 'GD_b1_diff', 'GD_b2_diff', 'GD_g1_diff', 'GD_g2_diff', 'GPT', 'GPT_pupilsize', 'GPT_t1', 'GPT_t2', 'GPT_a1', 'GPT_a2', 'GPT_b1', 'GPT_b2', 'GPT_g1', 'GPT_g2', 'GPT_t1_diff', 'GPT_t2_diff', 'GPT_a1_diff', 'GPT_a2_diff', 'GPT_b1_diff', 'GPT_b2_diff', 'GPT_g1_diff', 'GPT_g2_diff', 'SFD', 'SFD_pupilsize', 'SFD_t1', 'SFD_t2', 

In [100]:
# example: get first word
print(word_data[0].content)

Presents


In [154]:
# example: get number of fixations of first word
print(word_data[0].nFixations)

4
